# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# Verify current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'
file_path_list = []
# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    # join the file path and roots with the subdirectories using glob
    curr_path = glob.glob(os.path.join(root,'*'))
    if ".ipynb_checkpoints" not in str(curr_path): # otherwise checkpoints will add duplicate data
        file_path_list += curr_path


/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = []
    
# for every filepath in the file path list 
for f in file_path_list:
    # reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 

print(len(full_data_rows_list))
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }""")
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

## Create queries to ask the following three questions of the data:

---

### 1. Return the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


#### Query 1:  Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

For this query, we can choose a composite key consisting of sessionid and iteminsession because we need to filter events according to sessionid and iteminsession.  

And we set **sessionid** as primary key(which will be used to decide in which node data will be saved) because it has more variance than **itemsession**(which is therefore used as clustering key to sort data within node)

In [8]:
try:
    session.execute("DROP TABLE IF EXISTS events_by_sessionid_iteminsession;")
    session.execute("""
    CREATE TABLE IF NOT EXISTS events_by_sessionid_iteminsession 
      ( 
         sessionid     INT,
         iteminsession INT,
         artist        TEXT, 
         length        FLOAT,
         song          TEXT,
         PRIMARY KEY(sessionid, iteminsession) 
      ); 
    """)
except Exception as e:
    print(e)


In [9]:
file = 'event_datafile_new.csv'
query = "INSERT INTO events_by_sessionid_iteminsession(sessionid, iteminsession, artist, length, song)"
query += " VALUES(%s, %s, %s, %s, %s);"

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        session.execute(query, (int(line[8]), int(line[3]), line[0], float(line[5]), line[9]))

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
pd.set_option('display.width', 512) # to display complete row in one line

query = "SELECT artist, song, length FROM events_by_sessionid_iteminsession where sessionId = 338 AND itemInSession = 4;"
try:
    df = pd.DataFrame(list(session.execute(query)))
    print(df[["artist", "song", "length"]])
except Exception as e:
    print(e)

      artist                             song      length
0  Faithless  Music Matters (Mark Knight Dub)  495.307312


### 2. Return only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

Here **(sessionid, userid, iteminsession)** is used as composite key. In which **sessionid** is primary key(good variance) and **userid** as clustering key. **iteminsession** is also used in clustering key because our results needs to be sorted by **itemInSession**.

In [11]:
try:
    session.execute("DROP TABLE IF EXISTS events_by_sessionid_userid;")
    session.execute("""
    CREATE TABLE IF NOT EXISTS events_by_sessionid_userid 
      ( 
         sessionid     INT,
         userid        INT,
         iteminsession INT,
         artist        TEXT,
         firstname     TEXT,
         lastname      TEXT, 
         song          TEXT,
         PRIMARY KEY(sessionid, userid, iteminsession)
      );
    """)
except Exception as e:
    print(e)

                    

In [12]:
file = 'event_datafile_new.csv'
query = "INSERT INTO events_by_sessionid_userid(sessionid, userid, iteminsession, artist, firstname, lastname, song)"
query += " VALUES(%s, %s, %s, %s, %s, %s, %s);"

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        session.execute(query, (int(line[8]), int(line[10]), int(line[3]), line[0], line[1], line[4], line[9]))

In [13]:
pd.set_option('display.width', 512) # to display complete row in one line
## Verify the data was entered into the table
query = "SELECT artist, song, firstname, lastname FROM events_by_sessionid_userid where userid = 10 AND sessionid = 182;"
try:
    df = pd.DataFrame(list(session.execute(query)))
    print(df[["artist", "song", "firstname", "lastname"]])
except Exception as e:
    print(e)

              artist                                               song firstname lastname
0   Down To The Bone                                 Keep On Keepin' On    Sylvie     Cruz
1       Three Drives                                        Greece 2000    Sylvie     Cruz
2  Sebastien Tellier                                          Kilometer    Sylvie     Cruz
3      Lonnie Gordon  Catch You Baby (Steve Pitron & Max Sanna Radio...    Sylvie     Cruz


### 3. Return every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

Along with **song** the **userid** is also included in composite key because multiple users could have listened to same song. Without **userid** the insert statement for **song** which is already in db would not add a new record, but rather would just update the current user.

In [14]:
try:
    session.execute("DROP TABLE IF EXISTS username_by_song;")
    session.execute("""
    CREATE TABLE IF NOT EXISTS username_by_song 
      ( 
         song          TEXT,
         userid        TEXT,
         firstname     TEXT,
         lastname      TEXT, 
         PRIMARY KEY(song, userid)
      );
    """)
except Exception as e:
    print(e)              

In [15]:
file = 'event_datafile_new.csv'
query = "INSERT INTO username_by_song(song, userid, firstname, lastname)"
query += " VALUES(%s, %s, %s, %s);"

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        session.execute(query, (line[9], line[10], line[1], line[4]))
        

In [16]:
pd.set_option('display.width', 512) # to display complete row in one line
## Verify the data was entered into the table
query = "SELECT firstname, lastname FROM username_by_song where song='All Hands Against His Own';"
try:
    df = pd.DataFrame(list(session.execute(query)))
    print(df[["firstname", "lastname"]])
except Exception as e:
    print(e)

    firstname lastname
0  Jacqueline    Lynch
1       Tegan   Levine
2        Sara  Johnson


### Drop the tables before closing out the sessions

In [17]:
## Drop the table before closing out the sessions

In [18]:
try:
    tables = ["events_by_sessionid_iteminsession", "events_by_sessionid_userid", "username_by_song"]
    for t in tables:
        session.execute(f"DROP TABLE IF EXISTS {t};")
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()